# IDS 576 - SP22 - Project - G2

In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import tensorflow as tf
# from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
f = open('intents.json','r')
data=json.load(f)

In [3]:
# questions=[]
# tags=[]
# for i,x in enumerate(data['intents']):
#     questions.append(x['patterns'])
#     tags.append(x['tag'])

In [4]:
words=[]
questions=[]
tags=[]
for intent in data['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        questions.append((w, intent['tag']))
        if intent['tag'] not in tags:
            tags.append(intent['tag'])

In [5]:
words = [lemmatizer.lemmatize(w.lower()) for w in words]
words = sorted(list(set(words)))
tags = sorted(list(set(tags)))

In [6]:
print (len(questions), "questions")
print (len(tags), "tags", tags)

47 questions
9 tags ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [7]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(tags,open('tags.pkl','wb'))

## Preparing the Data

In [8]:
training = []
output_empty = [0] * len(tags)
for doc in questions:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[tags.index(doc[1])] = 1
    training.append([bag, output_row])

In [9]:
training = np.array(training)

C:\Users\ZOHAIB~1\AppData\Local\Temp/ipykernel_4848/3578359385.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [10]:
train_X = list(training[:,0])
train_y = list(training[:,1])

In [11]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [57]:
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_X), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/200
10/10 [==============================] - 0s 887us/step - loss: 0.1403 - accuracy: 0.9574
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1381 - accuracy: 0.9574
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1761 - accuracy: 0.9149
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1092 - accuracy: 0.9574
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0948 - accuracy: 0.9787
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1220 - accuracy: 0.9787
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1638 - accuracy: 0.9362
Epoch 8/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1085 - accuracy: 1.0000
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1590 - accuracy: 0.9574
Epoch 10/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1015 - accuracy: 0.9787
Epoch 1

10/10 [==============================] - 0s 997us/step - loss: 0.0820 - accuracy: 0.9787
Epoch 83/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1196 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0987 - accuracy: 0.9787
Epoch 85/200
10/10 [==============================] - 0s 997us/step - loss: 0.0498 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 997us/step - loss: 0.1159 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 886us/step - loss: 0.0840 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 886us/step - loss: 0.1045 - accuracy: 0.9787
Epoch 89/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0914 - accuracy: 0.9574
Epoch 90/200
10/10 [==============================] - 0s 997us/step - loss: 0.0508 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 997us/step - loss: 0.0589 - accuracy: 0.9574

10/10 [==============================] - 0s 997us/step - loss: 0.1804 - accuracy: 0.9149
Epoch 163/200
10/10 [==============================] - 0s 998us/step - loss: 0.0527 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0927 - accuracy: 0.9787
Epoch 165/200
10/10 [==============================] - 0s 997us/step - loss: 0.0827 - accuracy: 0.9787
Epoch 166/200
10/10 [==============================] - 0s 997us/step - loss: 0.0763 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 886us/step - loss: 0.1199 - accuracy: 0.9574
Epoch 168/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0987 - accuracy: 0.9787
Epoch 169/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1728 - accuracy: 0.9362
Epoch 170/200
10/10 [==============================] - 0s 997us/step - loss: 0.0825 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 997us/step - loss: 0.0575 - accurac